In [ ]:
import pandas as pd
import numpy as np

# Data Cleaning & Manipulation
With Pandas, we can do more than just select data that is already there. 
We can add new columns to our datasets, apply functions, iterate thorugh each row in the dataframe, and more.

This is where we move from "pandas for exploring our data" to "pandas for getting our data ready to feed into models".

In [ ]:
wine = pd.read_csv('data/wine_reviews/winemag-data_first150k.csv', index_col=0)
energy = pd.read_csv('data/energy/PJM_Load_hourly.csv', parse_dates=True, index_col=0)

### Dealing with Null Values:

In [ ]:
print('Does the wine df conatin nulls?:', wine.isnull().any().any())
print('Does the energy df contain nulls?:', energy.isnull().any().any())

In [ ]:
wine.isnull().any().any()

**Question:** why did we use ".any()" twice when asking this question?

Now, we need to deal with the null values in the wine table.
How we deal with them will depend on the data type of the column in which null values are found, and how we hope to use the data.

If a wine doesn't have a region listed, we can still get a lot of information about it from things like its country. But, if a wine doesn't have a price listed, it's difficult to get much information out of it. 

So, if a wine has a null value for either of its regions, we will set it to be "unknown". But, if it has a null price, we will drop the row.

In [ ]:
wine.isnull().any()

In [ ]:
print('Shape of wine table before is: ', wine.shape)

null_country = wine[wine['country'].isnull()]
print('Shape of null countries table is:', null_country.shape)

wine = wine[~wine.index.isin(null_country.index)]
print('Shape of wine table after is: ', wine.shape)

The 5 rows where a NaN value existed for country have been dropped.

Now we'll do the same for rows with a null price.

In [ ]:
print('Shape of wine table before is: ', wine.shape)

null_price = wine[wine['price'].isnull()]
print('Shape of null prices table is:', null_price.shape)

wine = wine[~wine.index.isin(null_price.index)]
print('Shape of wine table after is: ', wine.shape)

Wow! We lost a lot of rows getting rid of the null price rows.
Can you think of another way we could deal with null prices that doesn't mean losing almost 1/10 of our data? What are the pros and cons to using these other methods?

In [ ]:
print('Shape of wine table before is: ', wine.shape)

null_price = wine[wine['price'] == 'unknown']
print('Shape of null prices table is:', null_price.shape)

wine = wine[~wine.index.isin(null_price.index)]
print('Shape of wine table after is: ', wine.shape)

Now, for the other columns in which there are nulls, we can use `df.fillna()` to populate the null values with a value of our choosing. 

This is where we have to make some decisions. Improperly setting variables can have unintended consequences on any analysis we plan to do later using this data.

In [ ]:
wine.isnull().any()

The only columns where we still have nulls are regions and designations.
To avoid having to drop out more rows, we will use "fillna()".

In [ ]:
wine.fillna('unknown', inplace=True)

We now have no null values in our dataset!

In [ ]:
wine.isnull().any()

In [ ]:
wine.head()

### Creating new columns from existing ones
In pandas, it's easy to make a new column from existing ones. 

In [ ]:
wine.loc[:, 'Region'] = wine.loc[:, 'region_1'] + '_' + wine.loc[:, 'region_2']

#### Using a function + apply to make a new column:
Let's say we want to make a new column with an ID we can use to identify the records.
We can use a helper function + the apply method in pandas to apply it to all rows.
Apply looks through each row or column of a dataframe (depending on Axis set).

In [ ]:
def add_unique_id(row):
    row['Unique ID'] = (row['country'] + "_" + 
                        row['winery'] + "_" + 
                        str(np.random.randint(low=1, high=1000)))
    return row

In [ ]:
wine = wine.apply(add_unique_id, axis=1)

In [ ]:
wine.head()

We can use the same approach to make a column for "great and cheap" wines, defined here as $50 or less and with a score >95.

In [ ]:
def helper(row):
    if (row['price'] < 50) & (row['points'] >=95):
        x = 1
    else:
        x = 0
    return x

In [ ]:
wine.price = wine.price.astype('int')

In [ ]:
wine['great and cheap'] = wine.apply(helper, axis=1)

We can then use that boolean value to sort on the wines.

In [ ]:
wine[wine['great and cheap']==1]

**Exercise:** Using a function, make a column that indicates if a wine comes from Africa.

**Exercise:** Make a new column in the energy dataset, showing the energy in time T-1 for each time T. Hint: there is a built-in method in Pandas that can help with this, no complicated functions needed :)

**Exercise:** Re-read the CSV for the wine data, and write a function to take care of all the nan values. Bonus points if it includes a test.